In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
import math

In [2]:
# load training and test datasets
test_set = pd.read_csv('../input/test_set.csv', parse_dates=[2,])
train_set = pd.read_csv('../input/train_set.csv', parse_dates=[2,])
tube              = pd.read_csv('../input/tube.csv', parse_dates=True)
specs             = pd.read_csv('../input/specs.csv', parse_dates=[2,])
bill_of_materials = pd.read_csv('../input/bill_of_materials.csv', parse_dates=True)
comp_adaptor      = pd.read_csv('../input/comp_adaptor.csv', parse_dates=[2,])
comp_boss         = pd.read_csv('../input/comp_boss.csv', parse_dates=[2,])
comp_elbow        = pd.read_csv('../input/comp_elbow.csv', parse_dates=[2,])
comp_float        = pd.read_csv('../input/comp_float.csv', parse_dates=[2,])
comp_hfl          = pd.read_csv('../input/comp_hfl.csv', parse_dates=[2,])
comp_nut          = pd.read_csv('../input/comp_nut.csv', parse_dates=[2,])
comp_other        = pd.read_csv('../input/comp_other.csv', parse_dates=[2,])
comp_sleeve       = pd.read_csv('../input/comp_sleeve.csv', parse_dates=[2,])
comp_straight     = pd.read_csv('../input/comp_straight.csv', parse_dates=[2,])
comp_tee          = pd.read_csv('../input/comp_tee.csv', parse_dates=[2,])
comp_threaded     = pd.read_csv('../input/comp_threaded.csv', parse_dates=[2,])
components        = pd.read_csv('../input/components.csv', parse_dates=[2,])
test_set          = pd.read_csv('../input/test_set.csv', parse_dates=[2,])
train_set         = pd.read_csv('../input/train_set.csv', parse_dates=[2,])
tube_end_form     = pd.read_csv('../input/tube_end_form.csv', parse_dates=True)
tube              = pd.read_csv('../input/tube.csv', parse_dates=True)
type_component    = pd.read_csv('../input/type_component.csv', parse_dates=True)
type_connection   = pd.read_csv('../input/type_connection.csv', parse_dates=True)
type_end_form     = pd.read_csv('../input/type_end_form.csv', parse_dates=True)


In [3]:
labels = train_set.cost.values
label_log = np.log1p(labels)

In [4]:
BoF_1 = bill_of_materials[['tube_assembly_id','component_id_1','quantity_1']]
BoF_2 = bill_of_materials[['tube_assembly_id','component_id_2','quantity_2']]
comp_adaptor = pd.merge(comp_adaptor,type_component,on='component_type_id',how='inner')
comp_boss = pd.merge(comp_boss,type_component,on='component_type_id',how='inner')
comp_elbow = pd.merge(comp_elbow,type_component,on='component_type_id',how='inner')
comp_float = pd.merge(comp_float,type_component,on='component_type_id',how='inner')
comp_hfl = pd.merge(comp_hfl,type_component,on='component_type_id',how='inner')
comp_nut  = pd.merge(comp_nut,type_component,on='component_type_id',how='inner')
comp_sleeve  = pd.merge(comp_sleeve,type_component,on='component_type_id',how='inner')
comp_straight  = pd.merge(comp_sleeve,type_component,on='component_type_id',how='inner')
comp_tee  = pd.merge(comp_straight,type_component,on='component_type_id',how='inner')
comp_threaded  = pd.merge(comp_threaded,type_component,on='component_type_id',how='inner')


In [5]:
def merge_test_train_spec_tube(df):
    df = pd.merge(df,tube,on='tube_assembly_id')
    df = pd.merge(df,specs,on='tube_assembly_id')
    return df


In [6]:
train_set = merge_test_train_spec_tube(train_set)
test_set = merge_test_train_spec_tube(test_set)

In [7]:
test_set = test_set.drop(['spec10'], axis=1)
train_set = train_set.drop(['spec10'], axis=1)
bill_of_materials = bill_of_materials.drop(['component_id_5','quantity_5','component_id_6',
                                            'quantity_6', 'component_id_7','quantity_7', 'component_id_8','quantity_8'], axis=1)
comp_threaded = comp_threaded.drop(['end_form_id_3', 'connection_type_id_3','length_3','thread_size_3',
                                   'thread_pitch_3', 'nominal_size_3', 'end_form_id_4','connection_type_id_4',
                                   'length_4','thread_size_4','thread_pitch_4', 'nominal_size_4'],axis=1)

In [8]:
train_set = pd.merge(train_set,bill_of_materials,on='tube_assembly_id',how='inner')
test_set = pd.merge(test_set,bill_of_materials,on='tube_assembly_id',how='inner')

In [9]:
components = components.rename({'name':'name_components'}, axis=1)
components = components.drop(['component_type_id'], axis=1)

In [10]:
comp_adaptor  = pd.merge(comp_adaptor,components,on='component_id',how='left')
comp_boss  = pd.merge(comp_boss,components,on='component_id',how='left')
comp_elbow  = pd.merge(comp_elbow,components,on='component_id',how='left')
comp_float  = pd.merge(comp_float,components,on='component_id',how='left')
comp_hfl  = pd.merge(comp_hfl,components,on='component_id',how='left')
comp_nut  = pd.merge(comp_nut,components,on='component_id',how='left')
comp_sleeve = pd.merge(comp_sleeve,components,on='component_id',how='left')

In [11]:
comp_straight = comp_straight.drop(['name_y'],axis=1)
comp_straight = comp_straight.rename({'name_x':'name'}, axis=1)
comp_straight = pd.merge(comp_straight,components,on='component_id',how='left')

In [12]:
comp_tee = comp_tee.drop(['name_y','name_x'],axis=1)
comp_tee = pd.merge(comp_tee,components,on='component_id',how='left')

In [13]:
comp_threaded = pd.merge(comp_threaded,components,on='component_id',how='left')

In [14]:
type_end_form = type_end_form.rename({'name':'name_end_form_id'}, axis=1)

In [15]:
type_connection = type_connection.rename({'name':'name_type_connection_id'}, axis=1)

In [16]:
comp_threaded = pd.merge(comp_threaded,type_end_form,left_on='end_form_id_1',right_on='end_form_id',how='inner')
comp_threaded = comp_threaded.rename({'name_end_form_id':'name_end_form_id_1'}, axis=1)
comp_threaded = comp_threaded.drop(['end_form_id'],axis=1)
comp_threaded = pd.merge(comp_threaded,type_connection,left_on='connection_type_id_1',
                         right_on='connection_type_id',how='inner')
comp_threaded = comp_threaded.rename({'name_type_connection_id':'name_type_connection_id_1'}, axis=1)
comp_threaded = comp_threaded.drop(['connection_type_id'],axis=1)
comp_threaded = comp_threaded[['component_id', 'component_type_id', 'adaptor_angle', 'overall_length',
       'hex_size', 'end_form_id_1','name_end_form_id_1', 'connection_type_id_1','name_type_connection_id_1','length_1',
       'thread_size_1', 'thread_pitch_1', 'nominal_size_1', 'end_form_id_2',
       'connection_type_id_2', 'length_2', 'thread_size_2', 'thread_pitch_2',
       'nominal_size_2', 'unique_feature', 'orientation', 'weight', 'name',
       'name_components']]

In [17]:
comp_adaptor = pd.merge(comp_adaptor,type_end_form,left_on='end_form_id_1',right_on='end_form_id',how='inner')
comp_adaptor = comp_adaptor.rename({'name_end_form_id':'name_end_form_id_1'}, axis=1)
comp_adaptor = comp_adaptor.drop(['end_form_id'],axis=1)
comp_adaptor = pd.merge(comp_adaptor,type_connection,left_on='connection_type_id_1',
                         right_on='connection_type_id',how='inner')
comp_adaptor = comp_adaptor.rename({'name_type_connection_id':'name_type_connection_id_1'}, axis=1)
comp_adaptor = comp_adaptor.drop(['connection_type_id'],axis=1)
comp_adaptor = comp_adaptor[['component_id', 'component_type_id', 'adaptor_angle', 'overall_length',
       'end_form_id_1','name_end_form_id_1', 'connection_type_id_1','name_type_connection_id_1','length_1',
       'thread_size_1', 'thread_pitch_1', 'nominal_size_1', 'end_form_id_2',
       'connection_type_id_2', 'length_2', 'thread_size_2', 'thread_pitch_2',
       'nominal_size_2','hex_size', 'unique_feature', 'orientation', 'weight', 'name',
       'name_components']]

In [18]:
comp_boss = pd.merge(comp_boss,type_connection,left_on='connection_type_id',
                         right_on='connection_type_id',how='inner')
comp_boss = comp_boss[['component_id', 'component_type_id', 'type', 'connection_type_id', 'name_type_connection_id',
       'outside_shape', 'base_type', 'height_over_tube', 'bolt_pattern_long',
       'bolt_pattern_wide', 'groove', 'base_diameter', 'shoulder_diameter',
       'unique_feature', 'orientation', 'weight', 'name', 'name_components',
      ]]

In [19]:
comp_sleeve = pd.merge(comp_sleeve,type_connection,left_on='connection_type_id',
                         right_on='connection_type_id',how='inner')
comp_sleeve = comp_sleeve[['component_id', 'component_type_id', 'connection_type_id', 'name_type_connection_id','length',
       'intended_nut_thread', 'intended_nut_pitch', 'unique_feature',
       'plating', 'orientation', 'weight', 'name', 'name_components']]

In [20]:
comp_straight = pd.merge(comp_straight,type_connection,left_on='connection_type_id',
                         right_on='connection_type_id',how='inner')
comp_straight = comp_straight[['component_id', 'component_type_id', 'connection_type_id','name_type_connection_id', 'length',
       'intended_nut_thread', 'intended_nut_pitch', 'unique_feature',
       'plating', 'orientation', 'weight', 'name', 'name_components'
       ]]

In [21]:
comp_tee = pd.merge(comp_tee,type_connection,left_on='connection_type_id',
                         right_on='connection_type_id',how='inner')
comp_tee = comp_tee[['component_id', 'component_type_id', 'connection_type_id','name_type_connection_id', 'length',
       'intended_nut_thread', 'intended_nut_pitch', 'unique_feature',
       'plating', 'orientation', 'weight', 'name', 'name_components'
       ]]

In [22]:
bill_of_materials = bill_of_materials.drop(['component_id_3', 'quantity_3', 'component_id_4', 'quantity_4'],axis=1)

In [23]:
BoF_1 = bill_of_materials.drop(['component_id_2', 'quantity_2'],axis=1)
BoF_2 = bill_of_materials.drop(['component_id_1', 'quantity_1'],axis=1)

In [24]:
BoF_1_adaptor = pd.merge(BoF_1, comp_adaptor, left_on='component_id_1',right_on='component_id',how='right')
BoF_1_boss = pd.merge(BoF_1, comp_boss, left_on='component_id_1',right_on='component_id',how='right')
BoF_1_elbow = pd.merge(BoF_1, comp_elbow, left_on='component_id_1',right_on='component_id',how='right')
BoF_1_float = pd.merge(BoF_1, comp_float, left_on='component_id_1',right_on='component_id',how='right')
BoF_1_nut = pd.merge(BoF_1, comp_nut, left_on='component_id_1',right_on='component_id',how='right')
BoF_1_other = pd.merge(BoF_1, comp_other, left_on='component_id_1',right_on='component_id',how='right')
BoF_1_sleeve = pd.merge(BoF_1, comp_sleeve, left_on='component_id_1',right_on='component_id',how='right')
BoF_1_straight = pd.merge(BoF_1, comp_straight, left_on='component_id_1',right_on='component_id',how='right')
BoF_1_tee = pd.merge(BoF_1, comp_tee, left_on='component_id_1',right_on='component_id',how='right')
BoF_1_threaded = pd.merge(BoF_1, comp_threaded, left_on='component_id_1',right_on='component_id',how='right')

In [25]:
def concat_ordered_columns(frames, ax):
    columns_ordered = []
    for frame in frames:
        columns_ordered.extend(x for x in frame.columns if x not in columns_ordered)
    final_df = pd.concat(frames,ignore_index=True,axis=ax)    
    return final_df[columns_ordered]       


dfs = [BoF_1_adaptor,BoF_1_boss,BoF_1_elbow,BoF_1_float,BoF_1_float,BoF_1_nut,BoF_1_other,
       BoF_1_sleeve,BoF_1_straight,BoF_1_tee,BoF_1_threaded]
full_1 = concat_ordered_columns(dfs,0)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [26]:
full_1 = full_1[pd.notnull(full_1['tube_assembly_id'])]

In [27]:
BoF_2_adaptor = pd.merge(BoF_2, comp_adaptor, left_on='component_id_2',right_on='component_id',how='right')
BoF_2_boss = pd.merge(BoF_2, comp_boss, left_on='component_id_2',right_on='component_id',how='right')
BoF_2_elbow = pd.merge(BoF_2, comp_elbow, left_on='component_id_2',right_on='component_id',how='right')
BoF_2_float = pd.merge(BoF_2, comp_float, left_on='component_id_2',right_on='component_id',how='right')
BoF_2_nut = pd.merge(BoF_2, comp_nut, left_on='component_id_2',right_on='component_id',how='right')
BoF_2_other = pd.merge(BoF_2, comp_other, left_on='component_id_2',right_on='component_id',how='right')
BoF_2_sleeve = pd.merge(BoF_2, comp_sleeve, left_on='component_id_2',right_on='component_id',how='right')
BoF_2_straight = pd.merge(BoF_2, comp_straight, left_on='component_id_2',right_on='component_id',how='right')
BoF_2_tee = pd.merge(BoF_2, comp_tee, left_on='component_id_2',right_on='component_id',how='right')
BoF_2_threaded = pd.merge(BoF_2, comp_threaded, left_on='component_id_2',right_on='component_id',how='right')

In [28]:
dfs = [BoF_2_adaptor,BoF_2_boss,BoF_2_elbow,BoF_2_float,BoF_2_float,BoF_2_nut,BoF_2_other,
       BoF_2_sleeve,BoF_2_straight,BoF_2_tee,BoF_2_threaded]
full_2 = concat_ordered_columns(dfs,0)

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """


In [29]:
train_set = pd.merge(train_set, full_1, on='tube_assembly_id',how='left')
train_set = pd.merge(train_set, full_2, on='tube_assembly_id',how='left')

test_set = pd.merge(test_set, full_1, on='tube_assembly_id',how='left')
test_set = pd.merge(test_set, full_2, on='tube_assembly_id',how='left')

In [30]:
train_set = train_set.drop_duplicates()
test_set = test_set.drop_duplicates()

In [31]:
train_set['year'] = train_set.quote_date.dt.year
train_set['month'] = train_set.quote_date.dt.month
train_set['day'] = train_set.quote_date.dt.day

In [32]:
test_set['quote_date'] = pd.to_datetime(test_set['quote_date'])

In [33]:
test_set['quote_date'] = pd.to_datetime(test_set['quote_date'])

test_set['year'] = test_set.quote_date.dt.year
test_set['month'] = test_set.quote_date.dt.month
test_set['day'] = test_set.quote_date.dt.day

In [34]:

idx = test_set.id.values.astype(int)
test_set = test_set.drop(['id', 'tube_assembly_id', 'quote_date'], axis = 1)
labels = train_set.cost.values


train_set = train_set.drop(['quote_date', 'cost', 'tube_assembly_id'], axis = 1)

In [35]:
train_set.shape,test_set.shape

((30213, 150), (30235, 150))

In [36]:

for column in train_set:
    if train_set[column].isnull().any():
           if train_set[column].dtypes == 'object':
                values = train_set[column].value_counts().keys().tolist()
                counts = train_set[column].value_counts().tolist()
                if len(counts)!=0:
                    maxx = max(counts)
                    ind  = counts.index(maxx)
                    train_set[column] = train_set[column].fillna(values[ind])
                    
                    
for column in test_set:
    if test_set[column].isnull().any():
           if test_set[column].dtypes == 'object':
                values = test_set[column].value_counts().keys().tolist()
                counts = test_set[column].value_counts().tolist()
                if len(counts)!=0:
                    maxx = max(counts)
                    ind  = counts.index(maxx)
                    test_set[column] = test_set[column].fillna(values[ind])                

In [37]:

for column in train_set:
    if train_set[column].isnull().any():
        if train_set[column].dtypes != 'object':
            train_set[column] = train_set[column].fillna(train_set[column].mean())
           
for column in test_set:
    if test_set[column].isnull().any():
        if test_set[column].dtypes != 'object':
            test_set[column] = test_set[column].fillna(test_set[column].mean())
           
                                

In [38]:
for col in train_set.columns:
    if  train_set[col].dtypes == 'object' :
        lbl = preprocessing.LabelEncoder()
        lbl.fit(train_set[col].astype(str))
        train_set[col] = lbl.transform(train_set[col].astype(str))

In [39]:
for col in test_set.columns:
    if  test_set[col].dtypes == 'object' :
        lbl = preprocessing.LabelEncoder()
        lbl.fit(test_set[col].astype(str))
        test_set[col] = lbl.transform(test_set[col].astype(str))

In [40]:
label_log = np.log1p(labels)

In [41]:
del BoF_1, BoF_1_adaptor,BoF_1_boss,BoF_1_elbow,BoF_1_float,BoF_1_nut,BoF_1_other,BoF_1_sleeve,BoF_1_straight,BoF_1_tee,BoF_1_threaded

In [42]:
del BoF_2, BoF_2_adaptor,BoF_2_boss,BoF_2_elbow,BoF_2_float,BoF_2_nut,BoF_2_other,BoF_2_sleeve,BoF_2_straight,BoF_2_tee,BoF_2_threaded

In [43]:
del bill_of_materials,comp_adaptor,comp_boss,comp_elbow,comp_float,comp_hfl,comp_nut,comp_other,comp_sleeve,comp_straight,comp_tee,comp_threaded,components

In [44]:
del full_1,full_2,specs,tube,tube_end_form,type_component,type_connection,type_end_form

In [45]:
del labels

In [46]:
ntrain=train_set.shape[0]
ntest=test_set.shape[0]
print(ntrain, ntest)



30213 30235


In [58]:
from sklearn.model_selection import KFold
# Some useful parameters which will come in handy later on
SEED = 4 # for reproducibility
NFOLDS = 5# set folds for out-of-fold prediction
kf = KFold(n_splits= NFOLDS, shuffle=False)
i=0
for train_index, test_index in kf.split(train_set):
    print(train[test_index])



[[3.600e+01 0.000e+00 0.000e+00 ... 2.013e+03 7.000e+00 7.000e+00]
 [3.600e+01 0.000e+00 0.000e+00 ... 2.013e+03 7.000e+00 7.000e+00]
 [3.600e+01 0.000e+00 0.000e+00 ... 2.013e+03 7.000e+00 7.000e+00]
 ...
 [3.900e+01 2.000e+00 0.000e+00 ... 2.012e+03 9.000e+00 1.100e+01]
 [3.900e+01 2.000e+00 0.000e+00 ... 2.012e+03 9.000e+00 1.100e+01]
 [3.600e+01 0.000e+00 0.000e+00 ... 2.013e+03 7.000e+00 1.000e+00]]
[[3.600e+01 0.000e+00 0.000e+00 ... 2.013e+03 7.000e+00 1.000e+00]
 [3.600e+01 0.000e+00 0.000e+00 ... 2.013e+03 7.000e+00 1.000e+00]
 [3.600e+01 0.000e+00 0.000e+00 ... 2.013e+03 7.000e+00 1.000e+00]
 ...
 [3.900e+01 4.500e+03 1.000e+00 ... 2.011e+03 6.000e+00 1.000e+00]
 [3.600e+01 1.000e+01 1.000e+00 ... 2.011e+03 8.000e+00 1.000e+00]
 [3.000e+01 9.180e+02 1.000e+01 ... 2.013e+03 1.000e+00 1.700e+01]]
[[3.000e+01 1.000e+00 2.000e+00 ... 2.013e+03 2.000e+00 6.000e+00]
 [9.000e+00 2.660e+02 9.000e+01 ... 2.011e+03 5.000e+00 1.000e+00]
 [9.000e+00 2.710e+02 8.400e+01 ... 2.011e+03 5.00

In [55]:

train = train_set.values
test = test_set.values

In [48]:
import xgboost as xgb
xgb_params = dict()
xgb_params['eta'] = 0.01
xgb_params['scale_pos_weight '] = 3
xgb_params['max_depth'] = 5
xgb_params['colsample_bytree'] = 0.8
xgb_params['colsample_bylevel'] = 1
xgb_params['colsample_bynode'] = 1
xgb_params['alpha'] = 5
xgb_params['lambda'] = 0.9
xgb_params['max_delta_step'] = 10
xgb_params['seed'] = 255

In [62]:
pred_train = np.zeros((ntrain,) )
score = 0
from sklearn import metrics
progress = {}
i = 1
for train_index, test_index in  kf.split(train_set):
  

    xgtrain = xgb.DMatrix(train[train_index],label_log[train_index])
    xg_val = xgb.DMatrix(train[test_index],label_log[test_index])
    xg_test = xgb.DMatrix(train[test_index])
    
    
    watchlist = [(xgtrain, 'train'),(xg_val,'val')]
    print('1000')
    num_rounds = 5000
    model = xgb.train(xgb_params, xgtrain, num_rounds,verbose_eval=1,evals=watchlist,evals_result=progress,early_stopping_rounds=100)
    
    

    model.save_model(str(i)+'xgb.model')
    pr=model.predict(xg_test,ntree_limit=model.best_ntree_limit)
    pred_train[test_index]=pr
    err=metrics.mean_squared_error(label_log[test_index], pr)
    score += err*(1/NFOLDS) 
    print("Fold", i, "Точность работы на тестовых данных:",  err)
    i+=1
    print(s)

1000
[0]	train-rmse:1.87515	val-rmse:1.86405
Multiple eval metrics have been passed: 'val-rmse' will be used for early stopping.

Will train until val-rmse hasn't improved in 100 rounds.
[1]	train-rmse:1.85748	val-rmse:1.84647
[2]	train-rmse:1.8403	val-rmse:1.82911
[3]	train-rmse:1.82333	val-rmse:1.81229
[4]	train-rmse:1.80619	val-rmse:1.79525
[5]	train-rmse:1.78938	val-rmse:1.77828
[6]	train-rmse:1.77399	val-rmse:1.76302
[7]	train-rmse:1.75735	val-rmse:1.74648
[8]	train-rmse:1.7409	val-rmse:1.7304
[9]	train-rmse:1.7246	val-rmse:1.71451
[10]	train-rmse:1.7085	val-rmse:1.69851
[11]	train-rmse:1.69281	val-rmse:1.68268
[12]	train-rmse:1.67713	val-rmse:1.66714
[13]	train-rmse:1.66283	val-rmse:1.653
[14]	train-rmse:1.64742	val-rmse:1.63778
[15]	train-rmse:1.63349	val-rmse:1.62408
[16]	train-rmse:1.61831	val-rmse:1.60901
[17]	train-rmse:1.6033	val-rmse:1.59406
[18]	train-rmse:1.58865	val-rmse:1.57937
[19]	train-rmse:1.57421	val-rmse:1.5648
[20]	train-rmse:1.55966	val-rmse:1.55032
[21]	train-

[190]	train-rmse:0.460533	val-rmse:0.468853
[191]	train-rmse:0.458346	val-rmse:0.46681
[192]	train-rmse:0.456208	val-rmse:0.464825
[193]	train-rmse:0.454399	val-rmse:0.463265
[194]	train-rmse:0.452197	val-rmse:0.461092
[195]	train-rmse:0.450422	val-rmse:0.459484
[196]	train-rmse:0.448315	val-rmse:0.457373
[197]	train-rmse:0.446615	val-rmse:0.455905
[198]	train-rmse:0.444677	val-rmse:0.454086
[199]	train-rmse:0.442694	val-rmse:0.452228
[200]	train-rmse:0.441001	val-rmse:0.45066
[201]	train-rmse:0.439059	val-rmse:0.448863
[202]	train-rmse:0.437183	val-rmse:0.447148
[203]	train-rmse:0.435298	val-rmse:0.4454
[204]	train-rmse:0.4336	val-rmse:0.44371
[205]	train-rmse:0.43178	val-rmse:0.442041
[206]	train-rmse:0.430092	val-rmse:0.440419
[207]	train-rmse:0.428217	val-rmse:0.438551
[208]	train-rmse:0.426457	val-rmse:0.436928
[209]	train-rmse:0.424717	val-rmse:0.435332
[210]	train-rmse:0.423257	val-rmse:0.434056
[211]	train-rmse:0.421579	val-rmse:0.432302
[212]	train-rmse:0.419907	val-rmse:0.430

[378]	train-rmse:0.302424	val-rmse:0.324021
[379]	train-rmse:0.302167	val-rmse:0.323814
[380]	train-rmse:0.301923	val-rmse:0.32364
[381]	train-rmse:0.30171	val-rmse:0.323506
[382]	train-rmse:0.301482	val-rmse:0.32336
[383]	train-rmse:0.301229	val-rmse:0.323166
[384]	train-rmse:0.300992	val-rmse:0.32299
[385]	train-rmse:0.300781	val-rmse:0.322812
[386]	train-rmse:0.300495	val-rmse:0.32266
[387]	train-rmse:0.300237	val-rmse:0.322461
[388]	train-rmse:0.300042	val-rmse:0.32232
[389]	train-rmse:0.299834	val-rmse:0.322193
[390]	train-rmse:0.299578	val-rmse:0.32199
[391]	train-rmse:0.299376	val-rmse:0.321823
[392]	train-rmse:0.299151	val-rmse:0.321613
[393]	train-rmse:0.298967	val-rmse:0.321462
[394]	train-rmse:0.298713	val-rmse:0.321354
[395]	train-rmse:0.298423	val-rmse:0.32117
[396]	train-rmse:0.298207	val-rmse:0.320987
[397]	train-rmse:0.298011	val-rmse:0.320872
[398]	train-rmse:0.297797	val-rmse:0.320744
[399]	train-rmse:0.297606	val-rmse:0.320581
[400]	train-rmse:0.297432	val-rmse:0.320

[566]	train-rmse:0.271533	val-rmse:0.303074
[567]	train-rmse:0.271414	val-rmse:0.303002
[568]	train-rmse:0.271316	val-rmse:0.302958
[569]	train-rmse:0.271229	val-rmse:0.302923
[570]	train-rmse:0.271081	val-rmse:0.302765
[571]	train-rmse:0.270961	val-rmse:0.302703
[572]	train-rmse:0.27086	val-rmse:0.302638
[573]	train-rmse:0.270759	val-rmse:0.302638
[574]	train-rmse:0.270638	val-rmse:0.302578
[575]	train-rmse:0.270499	val-rmse:0.302473
[576]	train-rmse:0.270318	val-rmse:0.302351
[577]	train-rmse:0.270206	val-rmse:0.302315
[578]	train-rmse:0.270142	val-rmse:0.302282
[579]	train-rmse:0.270028	val-rmse:0.302197
[580]	train-rmse:0.26991	val-rmse:0.302129
[581]	train-rmse:0.269816	val-rmse:0.302091
[582]	train-rmse:0.269713	val-rmse:0.302078
[583]	train-rmse:0.269635	val-rmse:0.302043
[584]	train-rmse:0.269541	val-rmse:0.301999
[585]	train-rmse:0.269372	val-rmse:0.301836
[586]	train-rmse:0.26925	val-rmse:0.30177
[587]	train-rmse:0.269146	val-rmse:0.301701
[588]	train-rmse:0.269036	val-rmse:0

[754]	train-rmse:0.253974	val-rmse:0.292135
[755]	train-rmse:0.25391	val-rmse:0.292095
[756]	train-rmse:0.253831	val-rmse:0.29207
[757]	train-rmse:0.25377	val-rmse:0.292046
[758]	train-rmse:0.253685	val-rmse:0.291986
[759]	train-rmse:0.25356	val-rmse:0.291899
[760]	train-rmse:0.253467	val-rmse:0.291839
[761]	train-rmse:0.253383	val-rmse:0.291811
[762]	train-rmse:0.253293	val-rmse:0.291747
[763]	train-rmse:0.253199	val-rmse:0.291678
[764]	train-rmse:0.253161	val-rmse:0.291661
[765]	train-rmse:0.253081	val-rmse:0.291632
[766]	train-rmse:0.252984	val-rmse:0.291554
[767]	train-rmse:0.252936	val-rmse:0.291535
[768]	train-rmse:0.252891	val-rmse:0.291486
[769]	train-rmse:0.252849	val-rmse:0.291479
[770]	train-rmse:0.252754	val-rmse:0.291426
[771]	train-rmse:0.252715	val-rmse:0.291418
[772]	train-rmse:0.25266	val-rmse:0.291377
[773]	train-rmse:0.252611	val-rmse:0.291358
[774]	train-rmse:0.252541	val-rmse:0.291326
[775]	train-rmse:0.252486	val-rmse:0.291288
[776]	train-rmse:0.252406	val-rmse:0.

...

[4183]	train-rmse:0.166284	val-rmse:0.272092
[4184]	train-rmse:0.166276	val-rmse:0.272095
[4185]	train-rmse:0.166263	val-rmse:0.272088
[4186]	train-rmse:0.166251	val-rmse:0.272077
[4187]	train-rmse:0.166232	val-rmse:0.272076
[4188]	train-rmse:0.166214	val-rmse:0.272068
[4189]	train-rmse:0.16621	val-rmse:0.272059
[4190]	train-rmse:0.166199	val-rmse:0.272067
[4191]	train-rmse:0.166191	val-rmse:0.272064
[4192]	train-rmse:0.166178	val-rmse:0.272057
[4193]	train-rmse:0.16616	val-rmse:0.272055
[4194]	train-rmse:0.166152	val-rmse:0.272059
[4195]	train-rmse:0.166138	val-rmse:0.272052
[4196]	train-rmse:0.166128	val-rmse:0.272049
[4197]	train-rmse:0.166117	val-rmse:0.272051
[4198]	train-rmse:0.166107	val-rmse:0.272041
[4199]	train-rmse:0.166094	val-rmse:0.272036
[4200]	train-rmse:0.166084	val-rmse:0.272041
[4201]	train-rmse:0.166079	val-rmse:0.272043
[4202]	train-rmse:0.166069	val-rmse:0.272047
[4203]	train-rmse:0.166059	val-rmse:0.272045
[4204]	train-rmse:0.166052	val-rmse:0.272043
[4205]	train

[4366]	train-rmse:0.164292	val-rmse:0.271856
[4367]	train-rmse:0.164277	val-rmse:0.271863
[4368]	train-rmse:0.164263	val-rmse:0.27186
[4369]	train-rmse:0.164254	val-rmse:0.271864
[4370]	train-rmse:0.164239	val-rmse:0.271862
[4371]	train-rmse:0.164231	val-rmse:0.271862
[4372]	train-rmse:0.16422	val-rmse:0.271858
[4373]	train-rmse:0.164212	val-rmse:0.27186
[4374]	train-rmse:0.164199	val-rmse:0.271857
[4375]	train-rmse:0.164191	val-rmse:0.27186
[4376]	train-rmse:0.164182	val-rmse:0.27186
[4377]	train-rmse:0.164167	val-rmse:0.271858
[4378]	train-rmse:0.164157	val-rmse:0.271854
[4379]	train-rmse:0.16415	val-rmse:0.271846
[4380]	train-rmse:0.164143	val-rmse:0.271841
[4381]	train-rmse:0.164131	val-rmse:0.271838
[4382]	train-rmse:0.164119	val-rmse:0.271832
[4383]	train-rmse:0.164108	val-rmse:0.271832
[4384]	train-rmse:0.164093	val-rmse:0.271837
[4385]	train-rmse:0.164084	val-rmse:0.271832
[4386]	train-rmse:0.164073	val-rmse:0.271837
[4387]	train-rmse:0.164067	val-rmse:0.271837
[4388]	train-rms

[4550]	train-rmse:0.162473	val-rmse:0.271537
[4551]	train-rmse:0.162467	val-rmse:0.271531
[4552]	train-rmse:0.162453	val-rmse:0.271538
[4553]	train-rmse:0.162444	val-rmse:0.271538
[4554]	train-rmse:0.162423	val-rmse:0.271545
[4555]	train-rmse:0.162413	val-rmse:0.271546
[4556]	train-rmse:0.162405	val-rmse:0.271546
[4557]	train-rmse:0.162398	val-rmse:0.271544
[4558]	train-rmse:0.162389	val-rmse:0.271546
[4559]	train-rmse:0.162374	val-rmse:0.27154
[4560]	train-rmse:0.162368	val-rmse:0.271535
[4561]	train-rmse:0.162355	val-rmse:0.271532
[4562]	train-rmse:0.16234	val-rmse:0.271525
[4563]	train-rmse:0.162329	val-rmse:0.271528
[4564]	train-rmse:0.162319	val-rmse:0.271525
[4565]	train-rmse:0.162307	val-rmse:0.271516
[4566]	train-rmse:0.162298	val-rmse:0.271512
[4567]	train-rmse:0.162288	val-rmse:0.271508
[4568]	train-rmse:0.162279	val-rmse:0.271511
[4569]	train-rmse:0.162272	val-rmse:0.271513
[4570]	train-rmse:0.162267	val-rmse:0.271506
[4571]	train-rmse:0.16225	val-rmse:0.271505
[4572]	train-

[4734]	train-rmse:0.160684	val-rmse:0.27134
[4735]	train-rmse:0.160676	val-rmse:0.271346
[4736]	train-rmse:0.160667	val-rmse:0.271342
[4737]	train-rmse:0.16066	val-rmse:0.271343
[4738]	train-rmse:0.160652	val-rmse:0.271343
[4739]	train-rmse:0.160645	val-rmse:0.271335
[4740]	train-rmse:0.16064	val-rmse:0.271325
[4741]	train-rmse:0.160629	val-rmse:0.271322
[4742]	train-rmse:0.160619	val-rmse:0.271324
[4743]	train-rmse:0.160612	val-rmse:0.271323
[4744]	train-rmse:0.160602	val-rmse:0.271328
[4745]	train-rmse:0.160595	val-rmse:0.271326
[4746]	train-rmse:0.160586	val-rmse:0.27132
[4747]	train-rmse:0.160578	val-rmse:0.271324
[4748]	train-rmse:0.160569	val-rmse:0.27133
[4749]	train-rmse:0.160558	val-rmse:0.271322
[4750]	train-rmse:0.16055	val-rmse:0.271322
[4751]	train-rmse:0.160543	val-rmse:0.271318
[4752]	train-rmse:0.160532	val-rmse:0.271317
[4753]	train-rmse:0.160525	val-rmse:0.271314
[4754]	train-rmse:0.16052	val-rmse:0.271307
[4755]	train-rmse:0.160514	val-rmse:0.271309
[4756]	train-rmse

[4918]	train-rmse:0.159252	val-rmse:0.271168
[4919]	train-rmse:0.159245	val-rmse:0.27117
[4920]	train-rmse:0.159238	val-rmse:0.271164
[4921]	train-rmse:0.159232	val-rmse:0.271166
[4922]	train-rmse:0.159225	val-rmse:0.271161
[4923]	train-rmse:0.15922	val-rmse:0.271154
[4924]	train-rmse:0.159208	val-rmse:0.271142
[4925]	train-rmse:0.159204	val-rmse:0.271144
[4926]	train-rmse:0.159199	val-rmse:0.271151
[4927]	train-rmse:0.159195	val-rmse:0.271154
[4928]	train-rmse:0.15919	val-rmse:0.271158
[4929]	train-rmse:0.159182	val-rmse:0.27115
[4930]	train-rmse:0.159176	val-rmse:0.271146
[4931]	train-rmse:0.159169	val-rmse:0.27114
[4932]	train-rmse:0.159163	val-rmse:0.271145
[4933]	train-rmse:0.15916	val-rmse:0.27115
[4934]	train-rmse:0.159153	val-rmse:0.271149
[4935]	train-rmse:0.159147	val-rmse:0.271143
[4936]	train-rmse:0.15914	val-rmse:0.271139
[4937]	train-rmse:0.159134	val-rmse:0.27113
[4938]	train-rmse:0.159125	val-rmse:0.271127
[4939]	train-rmse:0.159121	val-rmse:0.271134
[4940]	train-rmse:0

In [63]:
score

0.07662630025433838

In [68]:
wr=1./NFOLDS
predTest = np.zeros((ntest,))
i=1
xg_test = xgb.DMatrix(test_set)
for i in np.arange(NFOLDS):    
    i+=1
    print(str(i)+'xgb.model')
    s=str(i)+'xgb.model'
    bst = xgb.Booster({'nthread': 4})  # init model
    bst.load_model(s)  # load data
    

   
    predTr=bst.predict(xg_test)
    predTest=predTest+predTr*wr
    

1xgb.model
2xgb.model
3xgb.model
4xgb.model
5xgb.model


In [72]:
preds = np.expm1(predTest)
preds

array([23.93765245, 13.53570918,  7.46981203, ...,  9.03056182,
       20.51575823, 25.58165413])

In [73]:
preds = pd.DataFrame({"id": idx, "cost": preds})
preds.to_csv('xgb.csv', index=False)